In [4]:
import os
import train_model
from aws_helpers import create_repository, setup_elb, create_ecs_service, deregister_old_taskdefinitions
from keras_to_tfserving import convert_keras_to_tf_model

## Requirements
This code requires a couple of things:
- Installed requirements
- Installed Docker (https://docs.docker.com/install/linux/docker-ce/ubuntu/)
- Installed & configured AWS CLI (https://aws.amazon.com/cli/)
- AWS Elastic Load Balancer setup as an Application Load Balancer with a listener on 443 (and an SSL certificate)
- AWS Elastic Container Service cluster setup and configured

## Setting up a new Service

In [5]:
version = 1
service_path = 'services.dev.sellpy.net'
service_name = "stockholm-ai"
cluster = "microservices"
env = "dev"
load_balancer_name = "sellpy-services"

### Train model & Convert model

In [8]:
model_name = "stockholm_ai_mnist.m"  

if model_name not in os.listdir("."):
    train_model.output_model(model_name, epochs=1)

!python3 keras_to_tfserving.py stockholm_ai_mnist.m models/stockholm-ai

# TODO: Make this repeatable in notebook
#convert_keras_to_tf_model(model_name=model_name,
#                          model_path="models/stockholm-ai", # Note that the model path is referred to in server.conf
#                          sequential=True)

/home/maxber/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
['stockholm_ai_mnist.m', 'models/stockholm-ai']
2018-03-26 14:22:29.100968: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loading Sequential model,
                 specify sequential=False to load functional model
inputs {
  key: "input_0"
  value {
    name: "conv2d_1_input_1:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "output_0"
  value {
    name: "dense_2_1/Softmax:0"
    dtype: 

/home/maxber/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
['stockholm_ai_mnist.m', 'models/stockholm-ai']
2018-03-26 14:20:54.425032: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Loading Sequential model,
                 specify sequential=False to load functional model
Traceback (most recent call last):
  File "keras_to_tfserving.py", line 86, in <module>
    convert_keras_to_tf_model(*sys.argv[1:])
  File "keras_to_tfserving.py", line 56, in convert_keras_to_tf_model
    builder = saved_model_builder.SavedModelBuilder(export_path)
  File "/home/maxber/.local/lib/python3.6/site-packages/tensorflow/python/save

### Create a repository on ECR, build & push the image

In [5]:
# Create an image repository for storing docker files.
create_repository(service_name)

# Shellscript.
# could write it like: subprocess.call("<command>", shell=True)
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev


Login Succeeded
Sending build context to Docker daemon  106.7MB
Step 1/10 : FROM ubuntu:16.04
 ---> dd6f76d9cc90
Step 2/10 : RUN apt-get update && apt-get install -y         build-essential         curl         git         &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 7f570d028e7e
Step 3/10 : ADD install.sh /tmp/install.sh
 ---> Using cache
 ---> 1aeab0fa1f4a
Step 4/10 : RUN sh -e /tmp/install.sh
 ---> Using cache
 ---> cc6955b1f03a
Step 5/10 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> 6b88877f44ae
Step 6/10 : RUN pip3 install -r /tmp/requirements.txt
 ---> Using cache
 ---> 606c5352e80f
Step 7/10 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y
 ---> Running in 886c838c3cfb
gpg: keyring `/tmp/tmp48fg18_i/secring.gpg' created
gpg: keyring `/tmp/tmp48fg18_i/pubring.gpg' created
gpg: requesting key BA9EF27F from hkp server keyserver.ubuntu.com
gpg: /tmp/tmp48fg18_i/trustdb.gpg: trustdb created
gpg: key BA9EF27F: public key "Launchpad Too

Setting up systemd-sysv (229-4ubuntu21.2) ...
(Reading database ... 26035 files and directories currently installed.)
Preparing to unpack .../libudev1_229-4ubuntu21.2_amd64.deb ...
Unpacking libudev1:amd64 (229-4ubuntu21.2) over (229-4ubuntu21.1) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
Setting up libudev1:amd64 (229-4ubuntu21.2) ...
Processing triggers for libc-bin (2.23-0ubuntu10) ...
(Reading database ... 26035 files and directories currently installed.)
Preparing to unpack .../isc-dhcp-client_4.3.3-5ubuntu12.10_amd64.deb ...
Unpacking isc-dhcp-client (4.3.3-5ubuntu12.10) over (4.3.3-5ubuntu12.9) ...
Preparing to unpack .../isc-dhcp-common_4.3.3-5ubuntu12.10_amd64.deb ...
Unpacking isc-dhcp-common (4.3.3-5ubuntu12.10) over (4.3.3-5ubuntu12.9) ...
Preparing to unpack .../curl_7.47.0-1ubuntu2.7_amd64.deb ...
Unpacking curl (7.47.0-1ubuntu2.7) over (7.47.0-1ubuntu2.6) ...
Preparing to unpack .../libcurl3-gnutls_7.47.0-1ubuntu2.7_amd64.deb ...
Unpacking libcurl3-gnutls:

Preparing to unpack .../libgfortran-5-dev_5.4.1-2ubuntu1~16.04_amd64.deb ...
Unpacking libgfortran-5-dev:amd64 (5.4.1-2ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../libgomp1_7.2.0-1ubuntu1~16.04_amd64.deb ...
Unpacking libgomp1:amd64 (7.2.0-1ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../libitm1_7.2.0-1ubuntu1~16.04_amd64.deb ...
Unpacking libitm1:amd64 (7.2.0-1ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../libatomic1_7.2.0-1ubuntu1~16.04_amd64.deb ...
Unpacking libatomic1:amd64 (7.2.0-1ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../libasan2_5.4.1-2ubuntu1~16.04_amd64.deb ...
Unpacking libasan2:amd64 (5.4.1-2ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../liblsan0_7.2.0-1ubuntu1~16.04_amd64.deb ...
Unpacking liblsan0:amd64 (7.2.0-1ubuntu1~16.04) over (5.4.0-6ubuntu1~16.04.9) ...
Preparing to unpack .../libtsan0_7.2.0-1ubuntu1~16.04_amd64.deb ...
Unpacking li

1bb2892c: Pushing  155.9MB/229.1MB11APushing  35.38MB/801MBPushing  70.96MB/229.1MBPushing    139MB/229.1MB10f293d: Pushing  235.1MB/1.097GB

10f293d: Pushed    1.11GB/1.097GBPushing  655.8MB/801MBPushing  873.5MB/1.097GBdev: digest: sha256:309fdb28bba95e0f62fd5d17f87341b841c0c0af1dee2a38b420da18c7ff2143 size: 3247


### Register a target & listener in the load balancer

In [6]:
target_group_arn = setup_elb(
    service=service_name,
    version=version,
    load_balancer_name=load_balancer_name,
    service_path=service_path
)

### Register a task definition and create an ECS service

In [8]:
# Create / Update a task definition.
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1

create_ecs_service(
    cluster=cluster,
    service=service_name,
    task_definition_family=service_name,
    target_group_arn=target_group_arn,
    service_count=1
)

deregister_old_taskdefinitions(service_name)

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:eu-west-1:966836717103:task-definition/stockholm-ai:20",
        "containerDefinitions": [
            {
                "name": "stockholm-ai",
                "image": "966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev",
                "cpu": 0,
                "memoryReservation": 300,
                "links": [
                    "serve_tensorflow_sthlm_ai:serve_tensorflow"
                ],
                "portMappings": [
                    {
                        "containerPort": 8080,
                        "hostPort": 1912,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "command": [
                    "python3",
                    "server.py"
                ],
                "environment": [
                    {
                        "name": "MNIST_SECRET_KEY",
                        "value": "sto

## Update existing services

In [12]:
#rebuild image
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev

# Register a new Task def 
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1
# Update the service with a new task def
!aws ecs update-service --service stockholm-ai --task-definition stockholm-ai --region eu-west-1  --cluster microservices
# Deregister tasks
deregister_old_taskdefinitions(service_name)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
Sending build context to Docker daemon     34MB
Step 1/10 : FROM ubuntu:16.04
 ---> f975c5035748
Step 2/10 : RUN apt-get update && apt-get install -y         build-essential         curl         git         &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 1b8db2213203
Step 3/10 : ADD install.sh /tmp/install.sh
 ---> Using cache
 ---> 2f8661714cd5
Step 4/10 : RUN sh -e /tmp/install.sh
 ---> Using cache
 ---> 705ffe62372f
Step 5/10 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> c5271e8a6b4e
Step 6/10 : RUN pip3 install -r /tmp/requirements.txt
 ---> Using cache
 ---> 3401bd12a327
Step 7/10 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y
 ---> Using cache
 ---> 2bfd47cd6b86
Step 8/10 : RUN apt-get update -y && apt-get upgrade -y && apt-get dist-upgrade -y
 ---> Using cache
 ---> 853316a548c1
Step 9/10 : WORKDIR /app
 ---> Using cache
 ---> 9764e2987889
Ste